In [52]:
import pandas as pd
import psycopg2
from reports.PRSchecker import PRS
from lib import PEchecker
from datetime import datetime
import warnings
import pandas.io.sql as sqlio
warnings.filterwarnings("ignore")
import re
import os 


#connect to the database
conn = psycopg2.connect(
        host='localhost',
        user='postgres',
        password='edsols',
        database='BravisaDB',  
        )

cur = conn.cursor()
# range of dates fr`om 2020-01-01 to 2023-11-23
date_range = pd.date_range(start='2020-01-01', end='2024-07-26')


# EPS

In [53]:
for curr_date in date_range:
    curr_date = curr_date.date()    
    # Define the SQL query with CTE to find the maximum "YearEnding" for each "CompanyCode" that is less than or equal to the provided date
    QuarterlyEPS_sql = """
    WITH MaxYearEnding AS (
        SELECT "CompanyCode", MAX("YearEnding") AS max_year
        FROM "QuarterlyEPS"
        WHERE "YearEnding" <= %s
        GROUP BY "CompanyCode"
    )
    SELECT q.*
    FROM "QuarterlyEPS" q
    JOIN MaxYearEnding m
    ON q."CompanyCode" = m."CompanyCode" AND q."YearEnding" = m.max_year
    """

    # Fetch the data using sqlio.read_sql_query and pass the current date as a parameter
    QuarterlyEPS = sqlio.read_sql_query(QuarterlyEPS_sql, params=(curr_date,), con=conn)

    QuarterlyEPS = QuarterlyEPS.drop_duplicates()

    # Define the SQL query with CTE to find the maximum "YearEnding" for each "CompanyCode" that is less than or equal to the provided date
    ttm_sql = """
    WITH MaxYearEnding AS (
        SELECT "CompanyCode", MAX("YearEnding") AS max_year
        FROM "TTM"
        WHERE "YearEnding" <= %s
        GROUP BY "CompanyCode"
    )
    SELECT t.*
    FROM "TTM" t
    JOIN MaxYearEnding m
    ON t."CompanyCode" = m."CompanyCode" AND t."YearEnding" = m.max_year
    """

    # Fetch the data using sqlio.read_sql_query and pass the current date as a parameter
    ttm_list = sqlio.read_sql_query(ttm_sql, params=(curr_date,), con=conn)
    ttm_list = ttm_list.drop_duplicates()


    EPS_sql = """SELECT * FROM "Reports"."EPS"
            WHERE "EPSDate"= %s
    """
    EPS = sqlio.read_sql_query(EPS_sql,params=(curr_date,),  con = conn)
    EPS = EPS.drop_duplicates()

    # merging Quarterly and TTM
    EPS_dashboard = pd.merge(QuarterlyEPS, ttm_list, how = 'outer', on='CompanyCode' )

    # Compile the regular expression pattern for efficiency
    x = re.compile("_x")
    y = re.compile("_y")

    columns = []

    # Iterate over each string in the list
    for column in EPS_dashboard.columns:
        # Use re.search() to look for the pattern in the string
        if x.search(column):
            renamed_col = column[:-2]
            # If a match is found, add the string to the matches list
            columns.append(renamed_col)
        elif y.search(column):
            renamed_col = column[:-2]+'_C'
            columns.append(renamed_col)
        else:
            columns.append(column)

    EPS_dashboard.columns = columns

    #merging EPS reports 
    EPS_dashboard = pd.merge(EPS_dashboard, EPS, how='right', on='CompanyCode')

    x = re.compile("_x")
    y = re.compile("_y")

    columns = []

    # Iterate over each string in the list
    for column in EPS_dashboard.columns:
        # Use re.search() to look for the pattern in the string
        if x.search(column):
            renamed_col = column[:-2]
            # If a match is found, add the string to the matches list
            columns.append(renamed_col)
        elif y.search(column):
            renamed_col = column[:-2]+'_EPS'
            columns.append(renamed_col)
        else:
            columns.append(column)

    EPS_dashboard.columns = columns

    EPS_dashboard[[ 'Sales', 'Expenses',
       'EBIDTA', 'Interest', 'Depreciation', 'Extraordinary', 'OPM', 'Tax',
       'PATRAW', 'PAT', 'Equity', 'Reserves', 'Sales_C', 'Expenses_C', 'EBIDTA_C', 'Interest_C',
       'Depreciation_C', 'Extraordinary_C', 'OPM_C', 'Tax_C', 'PAT_C',
       'Equity_C', 'Reserves_C']] = EPS_dashboard[[ 'Sales', 'Expenses',
       'EBIDTA', 'Interest', 'Depreciation', 'Extraordinary', 'OPM', 'Tax',
       'PATRAW', 'PAT', 'Equity', 'Reserves', 'Sales_C', 'Expenses_C', 'EBIDTA_C', 'Interest_C',
       'Depreciation_C', 'Extraordinary_C', 'OPM_C', 'Tax_C', 'PAT_C',
       'Equity_C', 'Reserves_C']]/10000000
    

    exportfilename = "EPS_dash.csv"
    exportfile = open(exportfilename, "w")
    EPS_dashboard.to_csv(exportfile, header=True, index=False,float_format="%.2f", lineterminator='\r')
    exportfile.close()

    copy_sql = """
        COPY "dash_process"."EPS" FROM stdin WITH CSV HEADER
        DELIMITER as ','
        """
    with open(exportfilename, 'r') as f:
        cur.copy_expert(sql=copy_sql, file=f)
        conn.commit()
        f.close()
    os.remove(exportfilename)
    print(curr_date, " finished")



    

2020-01-01  finished
2020-01-02  finished
2020-01-03  finished
2020-01-04  finished
2020-01-05  finished
2020-01-06  finished
2020-01-07  finished
2020-01-08  finished
2020-01-09  finished
2020-01-10  finished
2020-01-11  finished
2020-01-12  finished
2020-01-13  finished
2020-01-14  finished
2020-01-15  finished
2020-01-16  finished
2020-01-17  finished
2020-01-18  finished
2020-01-19  finished
2020-01-20  finished
2020-01-21  finished
2020-01-22  finished
2020-01-23  finished
2020-01-24  finished
2020-01-25  finished
2020-01-26  finished
2020-01-27  finished
2020-01-28  finished
2020-01-29  finished
2020-01-30  finished
2020-01-31  finished
2020-02-01  finished
2020-02-02  finished
2020-02-03  finished
2020-02-04  finished
2020-02-05  finished
2020-02-06  finished
2020-02-07  finished
2020-02-08  finished
2020-02-09  finished
2020-02-10  finished
2020-02-11  finished
2020-02-12  finished
2020-02-13  finished
2020-02-14  finished
2020-02-15  finished
2020-02-16  finished
2020-02-17  f

In [47]:
EPS_dashboard.columns

Index(['CompanyCode', 'YearEnding', 'Months', 'Quarter', 'Sales', 'Expenses',
       'EBIDTA', 'Interest', 'Depreciation', 'Extraordinary', 'OPM', 'Tax',
       'PATRAW', 'PAT', 'Equity', 'Reserves', 'EPS', 'NPM', 'Ext_Flag',
       'Q1 EPS Growth', 'Q1 Sales Growth', 'Q2 EPS', 'Q2 EPS Growth',
       'Q2 Sales', 'Q2 Sales Growth', 'E_ERS', 'YearEnding_C', 'Months_C',
       'Quarter_C', 'Sales_C', 'Expenses_C', 'EBIDTA_C', 'Interest_C',
       'Depreciation_C', 'Extraordinary_C', 'OPM_C', 'Tax_C', 'PAT_C',
       'Equity_C', 'Reserves_C', 'EPS_C', 'NPM_C', 'E_ERS_C', 'NSECode',
       'BSECode', 'CompanyName', 'ISIN', 'Months_EPS', 'Quarter_EPS',
       'YearEnding_EPS', 'Q1 EPS', 'Q1 EPS Growth_EPS', 'Q1 Sales',
       'Q1 Sales Growth_EPS', 'Q2 EPS_EPS', 'Q2 EPS Growth_EPS',
       'Q2 Sales_EPS', 'Q2 Sales Growth_EPS', 'TTM1 EPS Growth',
       'TTM1 Sales Growth', 'TTM2 EPS Growth', 'TTM2 Sales Growth',
       'TTM3 EPS Growth', 'TTM3 Sales Growth', 'NPM_EPS', 'EPS Rating',
      

In [84]:
conn.rollback()

In [45]:
for curr_date in date_range:
    curr_date = curr_date.date()    
    # Define the SQL query with CTE to find the maximum "YearEnding" for each "CompanyCode" that is less than or equal to the provided date
    QuarterlyEPS_sql = """
    WITH MaxYearEnding AS (
        SELECT "CompanyCode", MAX("YearEnding") AS max_year
        FROM "QuarterlyEPS"
        WHERE "YearEnding" <= %s
        GROUP BY "CompanyCode"
    )
    SELECT q.*
    FROM "QuarterlyEPS" q
    JOIN MaxYearEnding m
    ON q."CompanyCode" = m."CompanyCode" AND q."YearEnding" = m.max_year
    """

    # Fetch the data using sqlio.read_sql_query and pass the current date as a parameter
    QuarterlyEPS = sqlio.read_sql_query(QuarterlyEPS_sql, params=(curr_date,), con=conn)

    QuarterlyEPS = QuarterlyEPS.drop_duplicates()

    # Define the SQL query with CTE to find the maximum "YearEnding" for each "CompanyCode" that is less than or equal to the provided date
    ttm_sql = """
    WITH MaxYearEnding AS (
        SELECT "CompanyCode", MAX("YearEnding") AS max_year
        FROM "TTM"
        WHERE "YearEnding" <= %s
        GROUP BY "CompanyCode"
    )
    SELECT t.*
    FROM "TTM" t
    JOIN MaxYearEnding m
    ON t."CompanyCode" = m."CompanyCode" AND t."YearEnding" = m.max_year
    """

    # Fetch the data using sqlio.read_sql_query and pass the current date as a parameter
    ttm_list = sqlio.read_sql_query(ttm_sql, params=(curr_date,), con=conn)
    ttm_list = ttm_list.drop_duplicates()


    EPS_sql = """SELECT * FROM "Reports"."EPS"
            WHERE "EPSDate"= %s
    """
    EPS = sqlio.read_sql_query(EPS_sql,params=(curr_date,),  con = conn)
    EPS = EPS.drop_duplicates()

    

In [46]:
EPS_dashboard = pd.merge(QuarterlyEPS, ttm_list, how = 'outer', on='CompanyCode' )

In [47]:
import re

# Compile the regular expression pattern for efficiency
x = re.compile("_x")
y = re.compile("_y")

columns = []

# Iterate over each string in the list
for column in EPS_dashboard.columns:
    # Use re.search() to look for the pattern in the string
    if x.search(column):
        renamed_col = column[:-2]
        # If a match is found, add the string to the matches list
        columns.append(renamed_col)
    elif y.search(column):
        renamed_col = column[:-2]+'_C'
        columns.append(renamed_col)
    else:
        columns.append(column)

EPS_dashboard.columns = columns



In [48]:
EPS_dashboard.columns

Index(['CompanyCode', 'YearEnding', 'Months', 'Quarter', 'Sales', 'Expenses',
       'EBIDTA', 'Interest', 'Depreciation', 'Extraordinary', 'OPM', 'Tax',
       'PATRAW', 'PAT', 'Equity', 'Reserves', 'EPS', 'NPM', 'Ext_Flag',
       'Q1 EPS Growth', 'Q1 Sales Growth', 'Q2 EPS', 'Q2 EPS Growth',
       'Q2 Sales', 'Q2 Sales Growth', 'E_ERS', 'YearEnding_C', 'Months_C',
       'Quarter_C', 'Sales_C', 'Expenses_C', 'EBIDTA_C', 'Interest_C',
       'Depreciation_C', 'Extraordinary_C', 'OPM_C', 'Tax_C', 'PAT_C',
       'Equity_C', 'Reserves_C', 'EPS_C', 'NPM_C', 'E_ERS_C'],
      dtype='object')

In [49]:
EPS_dashboard = pd.merge(EPS_dashboard, EPS, how='right', on='CompanyCode')

In [50]:
import re
# Compile the regular expression pattern for efficiency
x = re.compile("_x")
y = re.compile("_y")

columns = []

# Iterate over each string in the list
for column in EPS_dashboard.columns:
    # Use re.search() to look for the pattern in the string
    if x.search(column):
        renamed_col = column[:-2]
        # If a match is found, add the string to the matches list
        columns.append(renamed_col)
    elif y.search(column):
        renamed_col = column[:-2]+'_EPS'
        columns.append(renamed_col)
    else:
        columns.append(column)

EPS_dashboard.columns = columns

In [76]:
import os 

exportfilename = "EPS_dash.csv"
exportfile = open(exportfilename, "w")
EPS_dashboard.to_csv(exportfile, header=True, index=False,float_format="%.2f", lineterminator='\r')
exportfile.close()

copy_sql = """
    COPY "dash_process"."EPS" FROM stdin WITH CSV HEADER
    DELIMITER as ','
    """
with open(exportfilename, 'r') as f:
    cur.copy_expert(sql=copy_sql, file=f)
    conn.commit()
    f.close()
os.remove(exportfilename)

In [75]:
conn.rollback()

In [73]:
for column in EPS_dashboard.columns:
    print(column,':',EPS_dashboard[column].dtype)

CompanyCode : float64
YearEnding : object
Months : float64
Quarter : float64
Sales : float64
Expenses : float64
EBIDTA : float64
Interest : float64
Depreciation : float64
Extraordinary : float64
OPM : float64
Tax : float64
PATRAW : float64
PAT : float64
Equity : float64
Reserves : float64
EPS : float64
NPM : float64
Ext_Flag : object
Q1 EPS Growth : float64
Q1 Sales Growth : float64
Q2 EPS : float64
Q2 EPS Growth : float64
Q2 Sales : float64
Q2 Sales Growth : float64
E_ERS : float64
YearEnding_C : object
Months_C : float64
Quarter_C : float64
Sales_C : float64
Expenses_C : float64
EBIDTA_C : float64
Interest_C : float64
Depreciation_C : float64
Extraordinary_C : float64
OPM_C : float64
Tax_C : float64
PAT_C : float64
Equity_C : float64
Reserves_C : float64
EPS_C : float64
NPM_C : float64
E_ERS_C : float64
NSECode : object
BSECode : float64
CompanyName : object
ISIN : object
Months_EPS : float64
Quarter_EPS : float64
YearEnding_EPS : object
Q1 EPS : float64
Q1 EPS Growth_EPS : float64
Q

# Summary 

In [12]:
import pandas as pd
import psycopg2
from reports.PRSchecker import PRS
from lib import PEchecker
from datetime import datetime
import warnings
import pandas.io.sql as sqlio
warnings.filterwarnings("ignore")
import re
import os 


#connect to the database
conn = psycopg2.connect(
        host='localhost',
        user='postgres',
        password='edsols',
        database='BravisaDB',  
        )

cur = conn.cursor()
# range of dates fr`om 2020-01-01 to 2023-11-23
date_range = pd.date_range(start='2024-08-19', end='2024-08-23')

In [13]:
for curr_date in date_range:
    curr_date = curr_date.date()    

    PRS_sql = """SELECT * FROM "Reports"."PRS"
            WHERE "Date"= %s
    """
    PRS = sqlio.read_sql_query(PRS_sql,params=(curr_date,),  con = conn)
    PRS = PRS.drop_duplicates()

    EPS_sql = """SELECT * FROM "Reports"."EPS"
            WHERE "EPSDate"= %s
    """
    EPS = sqlio.read_sql_query(EPS_sql,params=(curr_date,),  con = conn)
    EPS = EPS.drop_duplicates()

    SMR_sql ="""SELECT * FROM "Reports"."SMR"
            WHERE "SMRDate"= %s """
    SMR = sqlio.read_sql_query(SMR_sql,params=(curr_date,),  con = conn)
    SMR = SMR.drop_duplicates() 

    E_ERS_sql = """SELECT * FROM "Reports"."EERS"
            WHERE "EERSDate"= %s
    """
    E_ERS = sqlio.read_sql_query(E_ERS_sql,params=(curr_date,),  con = conn)
    E_ERS = E_ERS.drop_duplicates()

    IRS_sql = """SELECT * FROM "Reports"."IRS"
            WHERE "GenDate"= %s
    """
    IRS = sqlio.read_sql_query(IRS_sql,params=(curr_date,),  con = conn)
    IRS = IRS.drop_duplicates()
    PRS = PRS[['NSECode', 'Close', 'Change 52W', 'RS 52W', "CompanyCode", "Market Cap Value"]]
    SMR = SMR[[ "Industry","CompanyCode"]]
    EPS = EPS[['Ranking', "CompanyCode"]]
    E_ERS = E_ERS[['Ranking', "CompanyCode"]]
    IRS = IRS[['Index', "Rank", "MCap_Open_Index", "MCap_High_Index" , 'MCap_Low_Index', 'MCap_Close_Index']]

    Summary = pd.merge(PRS, EPS, on="CompanyCode", how='left')
    Summary = Summary.rename(columns={"Ranking": "EPS Rank"})

    Summary = pd.merge(Summary, E_ERS, on="CompanyCode", how='left')
    Summary = Summary.rename(columns={"Ranking": "EERS Rank"})

    Summary['CompanyCode'] = Summary['CompanyCode'].astype(float)
    SMR['CompanyCode'] = SMR['CompanyCode'].astype(float)

    Summary = pd.merge(Summary, SMR, on="CompanyCode", how='left')

        # Step 1: Create a dictionary from the IRS DataFrame
    rank_dict = IRS.set_index('Index')['Rank'].to_dict()
    open_dict = IRS.set_index('Index')['MCap_Open_Index'].to_dict()
    high_dict = IRS.set_index('Index')['MCap_High_Index'].to_dict()
    low_dict = IRS.set_index('Index')['MCap_Low_Index'].to_dict()
    close_dict = IRS.set_index('Index')['MCap_Close_Index'].to_dict()


        # Step 2: Map the dictionary values to the Summary DataFrame
    Summary['Rank'] = Summary['Industry'].map(rank_dict)
    Summary['Mcap_Open'] = Summary['Industry'].map(open_dict)
    Summary['Mcap_High'] = Summary['Industry'].map(high_dict)
    Summary['Mcap_Low'] = Summary['Industry'].map(low_dict)
    Summary['Mcap_Close'] = Summary['Industry'].map(close_dict)

    Summary['Date'] = curr_date
    
    exportfilename = "Summary.csv"
    exportfile = open(exportfilename, "w")
    Summary.to_csv(exportfile, header=True, index=False,float_format="%.2f", lineterminator='\r')
    exportfile.close()

    copy_sql = """
    COPY "dash_process"."Summary" FROM stdin WITH CSV HEADER
    DELIMITER as ','
    """
    with open(exportfilename, 'r') as f:
        cur.copy_expert(sql=copy_sql, file=f)
        conn.commit()
        f.close()
    os.remove(exportfilename)

    print("finished for ", curr_date)


finished for  2024-08-19
finished for  2024-08-20
finished for  2024-08-21
finished for  2024-08-22
finished for  2024-08-23


In [34]:
conn.rollback()